In [9]:
import pandas as pd 
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
target = train_data.SalePrice

# We use the simplest approach and drop the NA values
# since here we are concerned about categorical variables

cols_with_missing = [col for col in train_data.columns if train_data[col].isnull().any()]
candidate_train_predictors = train_data.drop(['Id', 'SalePrice'] + cols_with_missing, axis=1)
candidate_test_predictors = test_data.drop(['Id'] + cols_with_missing, axis=1)

low_cardinality_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].nunique() < 10 and
                                candidate_train_predictors[cname].dtype == "object"]

numeric_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].dtype in ['int64', 'float64']]
my_cols = low_cardinality_cols + numeric_cols
train_predictors = candidate_train_predictors[my_cols]
test_predictors = candidate_test_predictors[my_cols]

In [10]:
train_predictors.dtypes.sample(10)

LotConfig     object
BsmtFinSF1     int64
PoolArea       int64
MoSold         int64
MSSubClass     int64
LotShape      object
CentralAir    object
Condition1    object
Street        object
Utilities     object
dtype: object

In [12]:
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)

In [14]:
# investigate the effect of adding categorical variables to our model 
from sklearn.model_selection import cross_val_score 
from sklearn.ensemble import RandomForestRegressor

def get_mae(X, y):
    return -1 * cross_val_score(RandomForestRegressor(50),
                               X,y, scoring='neg_mean_absolute_error').mean()

predictors_without_categoricals = train_predictors.select_dtypes(exclude=['object'])

mae_without_categoricals = get_mae(predictors_without_categoricals, target)

mae_one_hot_encoded = get_mae(one_hot_encoded_training_predictors, target)

print('Mean Absolute Error when Dropping Categoricals:'
      + str(int(mae_without_categoricals)))
print('Mean Absolute Error with One-hot Encoding:'
     + str(int(mae_one_hot_encoded)))


Mean Absolute Error when Dropping Categoricals:18656
Mean Absolute Error with One-hot Encoding:18210


Scikit-learn is sensitive to the ordering of columns, so if the training dataset and test datasets get misaligned, the results will be nonsense! This could happen if a categorical had a different number of values in the training data vs the test data. 
Ensure the test data is encoded in the same manner as the training with the align command: 

In [16]:
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
one_hot_encoded_test_predictors = pd.get_dummies(test_predictors)

final_train, final_test = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors,
                                                                   join='left',
                                                                   axis=1)

## XGBoost 
It is an implementation of the Gradient Boosted Decision Trees algorithm. 

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
data = pd.read_csv('train.csv')
data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = data.SalePrice
X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])
train_X, test_X, train_y, test_y = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)

my_imputer = Imputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)


/home/anirudh/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [21]:
from xgboost import XGBRegressor
my_model = XGBRegressor()
my_model.fit(train_X, train_y, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [22]:
predictions = my_model.predict(test_X)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))

Mean Absolute Error : 15973.353306934932


## Model Tuning 

XGBoost has a few parameters that can dramatically affect our model's accuracy and training speed. 
 **n_estimators and early_stopping_rounds**
 **n_estimators** specifies how many times to go through the modelling cycle described above. 
 In the underfitting vs overfitting graph, n_estimators moves you further to the right. Too low a value causes underfitting. 
 
 The argument early_stopping_rounds offers a way to automatically find the ideal value. Early stopping causes the model to stop iterating when the validation score stops improving, even if we aren't at the hard stop for n_estimators. It;s smart to set a high value for n_estimators and then use early_stopping_rounds to find the optimal time to stop iterating. 

In [26]:
my_model = XGBRegressor(n_estimators=1000)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
            eval_set=[(test_X, test_y)], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In general, a small learning rate(and large number of estimators) will yield more accurate models, though it will also take longer to train since it does more iterations through the cycle. 

In [27]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)